# libraries

In [1]:
# Import necessary libraries

 # For interacting with the operating system
import os 

# For numerical operations on arrays
import numpy as np  

# For image processing
import cv2  

 # For generating random numbers
import random 

# For plotting and visualization
import matplotlib.pyplot as plt  


# Import specific functionalities from TensorFlow and Scikit-learn

 # For deep learning
import tensorflow as tf 

# For data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

# For converting labels to one-hot encoding
from tensorflow.keras.utils import to_categorical  

# For building and loading models
from tensorflow.keras.models import Sequential, load_model  

# For splitting data into training and validation sets
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# For building layers of the neural network
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout  


# Load Dataset

In [2]:
# Initialize lists to hold images and their corresponding labels
images = []
labels = []

# Loop through each label directory
for index, label in enumerate(['Closed', 'Open', 'no_yawn', 'yawn']):  # Assign numeric labels: 0, 1, 2, 3
    
    train_path = '/kaggle/input/drowsiness-dataset/train'
    
    images_folder_path = os.path.join(train_path, label)  # Construct the path to the image folder for the current label

    # Loop through all images in the current label directory
    for image_number in os.listdir(images_folder_path):
        # Construct the full path to the image
        image_path = os.path.join(images_folder_path, image_number)
        
        # Read the image using matplotlib
        image = plt.imread(image_path)
        
        # Resize the image to (150, 150) using OpenCV
        resized_image = cv2.resize(image, (150, 150))
        
        # Convert the image to grayscale
        image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY) 
        
        # Append the processed image and its label to the respective lists
        images.append(image)
        labels.append(index) # numeric labels

# Convert the lists of images and labels to NumPy arrays for efficient numerical operations
images = np.array(images)
labels = np.array(labels)

print(f"Images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")


Images shape: (2900, 150, 150)
Labels shape: (2900,)


# Split Data

In [ ]:
# Ensure the images have an additional channel dimension for compatibility with CNNs
images = np.expand_dims(images, axis=-1)

# Split the data into training and temporary sets (60% training, 40% temporary)
x_train, x_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.4, 
                                                    shuffle=True, random_state=42)

# Split the temporary set into validation and test sets (20% each from the original data)
x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, 
                                                shuffle=True, random_state=42)

# Convert the labels to one-hot encoded format for the neural network
y_train = to_categorical(y_train, num_classes=4)
y_val = to_categorical(y_val, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

# Print the shapes of the training, validation, and test sets
print(f'Train set: {x_train.shape}, {y_train.shape}') 
print(f'Validation set: {x_val.shape}, {y_val.shape}')
print(f'Test set: {x_test.shape}, {y_test.shape}')


# Data Augmentation

In [ ]:
# Import the necessary library for data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define custom contrast adjustment function
def random_contrast(image, lower=0.9, upper=1.1):
    return tf.image.random_contrast(image, lower=lower, upper=upper)

# Define custom preprocessing function
def custom_preprocess(image):
    image = random_contrast(image)
    return image

# Define ImageDataGenerators for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.9, 1.1],
    preprocessing_function=custom_preprocess
)

val_test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values for validation and test data to the range [0, 1]

# Create data generators for training, validation, and test sets
train_generator = train_datagen.flow(x_train, y_train, batch_size=32)  # Generate batches of tensor image data for training
val_generator = val_test_datagen.flow(x_val, y_val, batch_size=32)  # Generate batches of tensor image data for validation
test_generator = val_test_datagen.flow(x_test, y_test, batch_size=32)  # Generate batches of tensor image data for testing


# Model

In [ ]:
# Determine the input shape for the CNN based on the shape of the images
input_shape = images.shape[1:] 
print("Input shape:", input_shape)

# Define a function to create the CNN model
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),  # First convolutional layer
        MaxPooling2D(pool_size=(2, 2)),  # First max pooling layer
        Conv2D(64, (3, 3), activation='relu'),  # Second convolutional layer
        MaxPooling2D(pool_size=(2, 2)),  # Second max pooling layer
        Conv2D(128, (3, 3), activation='relu'),  # Third convolutional layer
        MaxPooling2D(pool_size=(2, 2)),  # Third max pooling layer
        Flatten(),  # Flatten the feature map to a 1D vector
        Dense(256, activation='relu'),  # Fully connected layer with 256 units
        Dropout(0.5),  # Dropout layer to prevent overfitting
        Dense(4, activation='softmax')  # Output layer with 4 units (one for each class)
    ])
    return model

# Create the CNN model using the defined function
model = create_model()

# Compile the model with Adam optimizer, categorical cross-entropy loss, and accuracy metric
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train The Model

In [ ]:
# Train the model using the training and validation data generators
history = model.fit(
    train_generator,  # Generator for training data
    epochs=50,  # Number of epochs to train the model
    validation_data=val_generator  # Generator for validation data
)


# Test the model

In [ ]:
# Evaluate the model using the test data generator
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))

# Print the test loss and accuracy
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


# Visualize training & validation accuracy and Loss Values

In [ ]:
# Import the necessary library for plotting
import matplotlib.pyplot as plt

# Create a figure with a specific size
plt.figure(figsize=(12, 4))

# Plot training and validation accuracy values
plt.subplot(1, 2, 1)  # Create a subplot (1 row, 2 columns, 1st plot)
plt.plot(history.history['accuracy'])  # Plot training accuracy
plt.plot(history.history['val_accuracy'])  # Plot validation accuracy
plt.title('Model Accuracy')  # Set the title of the plot
plt.xlabel('Epoch')  # Set the x-axis label
plt.ylabel('Accuracy')  # Set the y-axis label
plt.legend(['Train', 'Validation'], loc='upper left')  # Add a legend to differentiate training and validation accuracy

# Plot training and validation loss values
plt.subplot(1, 2, 2)  # Create a subplot (1 row, 2 columns, 2nd plot)
plt.plot(history.history['loss'])  # Plot training loss
plt.plot(history.history['val_loss'])  # Plot validation loss
plt.title('Model Loss')  # Set the title of the plot
plt.xlabel('Epoch')  # Set the x-axis label
plt.ylabel('Loss')  # Set the y-axis label
plt.legend(['Train', 'Validation'], loc='upper left')  # Add a legend to differentiate training and validation loss

# Adjust the layout to prevent overlap
plt.tight_layout()

# Display the plots
plt.show()


# Some Examples

# try model in random image 

In [ ]:
# Define the class labels dictionary
classes = {
    0: "Closed",
    1: "Open",
    2: "no_yawn",
    3: "yawn"
}

# Path to the image you want to predict
image_path = '/kaggle/input/drowsiness-dataset/train/Closed/_1.jpg'

# Read the image using matplotlib
image = plt.imread(image_path)

# Resize the image to (150,150) using OpenCV
resized_image = cv2.resize(image, (150, 150))

# Convert the image to grayscale
gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

# Add a channel dimension to the image
gray_image = np.expand_dims(gray_image, axis=-1)

# Add a batch dimension to the image
gray_image = np.expand_dims(gray_image, axis=0)

# Predict the class of the image
predictions = model.predict(gray_image)

# Convert predicted probabilities to class index
predicted_class = np.argmax(predictions, axis=1)[0]

# Retrieve the class label from the dictionary
predicted_label = classes[predicted_class]

print(f"Actual Class Closed")
print(f"Predicted Class: {predicted_label}")


# save model

In [ ]:
import pickle
import tensorflow as tf

# # Assuming 'model' is your trained TensorFlow model
# model.save('DDD-model.pkl')

model.save("DDD_model.h5")
print("Model saved as DDD_model.h5")

# End